In [1]:
import numpy as np
import pandas as pd
from sklearn import pipeline, preprocessing
from sklearn.pipeline import Pipeline, make_pipeline

from df_transformers import *

In [2]:
train_data = pd.read_csv("data/cleaned_train_data.csv", index_col="id")
test_data = pd.read_csv("data/cleaned_test_data.csv", index_col="id")
print(f"Shape of train data: {train_data.shape}. Shape of test data: {test_data.shape}")
train_size = train_data.shape[0]

Shape of train data: (600000, 24). Shape of test data: (400000, 23)


In [3]:
# Объединим датасеты для обработки
data = pd.concat([train_data.drop(['target'], axis=1), test_data])
y = train_data['target']

### Разделяем признаки по типу

In [4]:
# Бинарные
bin_features = ['bin_0', 'bin_1', 'bin_2', 'bin_3', 'bin_4']
# Категориальные
nom_features = ['nom_0', 'nom_1', 'nom_2', 'nom_3', 'nom_4']
# Хеш
hex_features = ['nom_5', 'nom_6', 'nom_7', 'nom_8', 'nom_9']
# Порядковые
ord_features = ['ord_0', 'ord_1', 'ord_2', 'ord_3', 'ord_4', 'ord_5']
ord_features_low_cardinality = ['ord_0', 'ord_1', 'ord_2']
ord_features_high_cardinality = ['ord_3', 'ord_4', 'ord_5']
# Цикличные
cyc_features = ['day', 'month']

### Собираем пайплайн

In [12]:
# Обработка бинарных признаков
bin_dict = {'T':1, 'F':0, 'Y':1, 'N':0}
bin_pipeline = make_pipeline(
    SelectColumnsTransformer(bin_features),
    BinaryColumnsTransformer(bin_dict = bin_dict),
)

# Обработка категориальных признаков с малым количеством категорий
nom_pipeline = make_pipeline(
    SelectColumnsTransformer(nom_features),
    DataFrameFunctionTransformer(lambda x: x.apply(str)),
    # Разобраться что за тип данных category в pd.DataFrame и что такое object_levels
    # DataFrameFunctionTransformer(lambda x:x.astype('category', categories=object_levels)),
    ToDummiesTransformer(),
    DropAllZeroTrainColumnsTransformer(),
)

# Обработка категориальных признаков с большим количеством категорий
hex_pipeline = make_pipeline(
    SelectColumnsTransformer(hex_features),
    DataFrameFunctionTransformer(lambda x: x.apply(str)),
    LabelTransformer(),
)

# Обработка порядковых признаков небольшим количеством категорий
# Возможно, для большей гибкости, необходимо заменить списки в списке на словари
ord_order = [
    [1.0, 2.0, 3.0],
    ['Novice', 'Contributor', 'Expert', 'Master', 'Grandmaster'],
    ['Freezing', 'Cold', 'Warm', 'Hot', 'Boiling Hot', 'Lava Hot']
]

ord_pipeline_low = make_pipeline(
    SelectColumnsTransformer(ord_features_low_cardinality),
    OrderedColumnsTransformer(order_list = ord_order),
)

# Обработка порядковых признаков большим количеством категорий
ord_pipeline_high = make_pipeline(
    SelectColumnsTransformer(ord_features_high_cardinality),
    OrderedColumnsTransformer(),
)

# Обработка цикличных признаков
cyc_pipeline = make_pipeline(
    SelectColumnsTransformer(cyc_features),
)

# Обединяем обработанные данные
preprocessing_features = DataFrameFeatureUnion([bin_pipeline, 
                                                nom_pipeline, 
                                                hex_pipeline, 
                                                ord_pipeline_low, 
                                                ord_pipeline_high, 
                                                cyc_pipeline])

In [13]:
prprd_data = preprocessing_features.fit_transform(data)

In [14]:
preprocessed_train_data = prprd_data.iloc[:train_size, :]
preprocessed_test_data = prprd_data.iloc[train_size:, :]

In [15]:
preprocessed_train_data['target'] = y

/Users/tarasovnm/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


### Обрабатываем и сохраняем данные

In [16]:
preprocessed_train_data.to_csv('data/preprocessed_train_data.csv', header=True)
preprocessed_test_data.to_csv('data/preprocessed_test_data.csv', header=True)

In [18]:
# Избранные ноутбуки из предыдущего соревнования
# https://www.kaggle.com/shahules/an-overview-of-encoding-techniques
# https://www.kaggle.com/kabure/eda-feat-engineering-encode-conquer
# https://www.kaggle.com/adaubas/2nd-place-solution-categorical-fe-callenge
# https://www.kaggle.com/abhishek/entity-embeddings-to-handle-categories
# https://www.kaggle.com/peterhurford/why-not-logistic-regression

# Посмотреть по текущему соревнованию
# https://www.kaggle.com/drcapa/categorical-feature-engineering-2-xgb
# https://www.kaggle.com/vikassingh1996/don-t-underestimate-the-power-of-a-logistic-reg

# https://habr.com/ru/company/ods/blog/326418/#label-encoding